## Adaptive Sampling Verification between:

* [Ruby's implementation](https://source.datanerd.us/ruby-agent/ruby_agent/blob/dev/lib/new_relic/agent/adaptive_sampler.rb)

* [Python's implementation](https://source.datanerd.us/python-agent/python_agent/blob/develop/newrelic/core/adaptive_sampler.py)



In [22]:
class BaseAgent(object):
    def __init__(self, last_transaction_count):
        self.sampled_count = 0
        self.max_sampled = 20
        self.sampling_target = 10
        self.max_ratio = 0.5
        self.transaction_count = last_transaction_count
        
    def intake_event(self, random_number):
        if self.sample(random_number):
            self.sampled_count += 1
            
    def sample(self, random_number):
        return False

In [23]:
class PythonAgent(BaseAgent):
    def __init__(self, last_transaction_count):
        super(PythonAgent, self).__init__(last_transaction_count)
        self.min_sampling_priority = 1 - self.sampling_target / last_transaction_count
    
    def sample(self, random_number):
        return self.compute_sampled(1 - random_number)
    
    def compute_sampled(self, priority):
        if self.sampled_count >= self.max_sampled:
            return False

        elif priority >= self.min_sampling_priority:
            self.sampled_count += 1

            # Determine if a backoff is required and compute it
            target = self.sampling_target
            if self.sampled_count > target:
                ratio = target / float(self.sampled_count)
                target = target ** ratio - target ** self.max_ratio
                target = max(0.0, target)

                self._update_min_priority(target)
            return True

        return False
        
    def _update_min_priority(self, target):
        if self.transaction_count > target:
            sampling_ratio = float(target) / self.transaction_count
            self.min_sampling_priority = 1.0 - sampling_ratio
        else:
            self.min_sampling_priority = 0.0

In [24]:
class RubyAgent(BaseAgent):
    def sample(self, random_number):
        return self.compute_sampled(self.transaction_count * random_number)
    
    def compute_sampled(self, random_number):
        target = self.sampling_target
        if self.sampled_count < target:
            return (random_number < target)
        else:
            ratio = target / self.sampled_count
            return random_number < (target ** ratio - target ** self.max_ratio)
        return False

In [27]:
import random

for throughput in [100, 200, 500, 1000, 2000, 20000, 200000]:
    p = PythonAgent(throughput)
    r = RubyAgent(throughput)
    
    for _ in range(throughput):
        rn = random.random()
        if p.intake_event(rn) != r.intake_event(rn):
            raise Exception("Error: agents do not match for throughput %d and input %f" % (throughput, rn))

If no exceptions are thrown, we're golden!